In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import os, os.path 
import numpy 
import pickle
from glob import glob


"""Change to the data folder"""
new_path = "data/new_train/"

# number of sequences in each dataset
# train:205942  val:3200 test: 36272 
# sequences sampled at 10HZ rate

In [2]:
class ArgoverseDataset(Dataset):
    """Dataset class for Argoverse"""
    def __init__(self, data_path: str, transform=None):
        super(ArgoverseDataset, self).__init__()
        self.data_path = data_path
        self.transform = transform

        self.pkl_list = glob(os.path.join(self.data_path, '*'))
        self.pkl_list.sort()
        
    def __len__(self):
        return len(self.pkl_list)

    def __getitem__(self, idx):

        pkl_path = self.pkl_list[idx]
        with open(pkl_path, 'rb') as f:
            data = pickle.load(f)
            
        if self.transform:
            data = self.transform(data)

        return data


# intialize a dataset
train_dataset  = ArgoverseDataset(data_path=new_path)

In [3]:
batch_sz = 100
import numpy as np

def my_collate_train(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
    batch_inp = []
    batch_out = []

    for scene in batch:
        agent = scene['agent_id']
        target = 0
        for x in range(len(scene['track_id'])):
            if scene['track_id'][x][0] == agent:
                target = x
        inp = [scene['p_in'][target], scene['v_in'][target]]
        out = [scene['p_out'][target], scene['v_out'][target]]
        batch_inp.append(inp)
        batch_out.append(out) 

    inp = torch.FloatTensor(batch_inp)
    out = torch.FloatTensor(batch_out)
    return [inp, out]

def my_collate_val(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """

    inp = [[scene['p_in'], scene['v_in']] for scene in batch]
    mask = [scene['car_mask'] for scene in batch]

    inp = torch.LongTensor(inp)
    mask = torch.LongTensor(mask)
    return [inp, mask]

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class Trajectory(nn.Module):

    def __init__(self):
        super(Trajectory, self).__init__()

        self.p_in = nn.Linear(2, 32)
        self.v_in = nn.Linear(2, 32)
        
        self.encoder = nn.LSTM(64, 64, 1)

        self.decoder_p = nn.LSTM(64, 128, 1)
        self.decoder_v = nn.LSTM(64, 128, 1)

        self.p_out = nn.Linear(128, 2)
        self.v_out = nn.Linear(128, 2)
                
    def forward(self, p, v):
        batch = p.shape[0]
        x_p = self.p_in(p)
        x_v = self.v_in(v)

        x = torch.cat((x_p, x_v), dim=2)
        x = x.permute(1, 0, 2)

        _, (state_h, _) = self.encoder(x)

        x = state_h.repeat(30, 1, 1)

        x_p, _ = self.decoder_p(x)
        x_v, _ = self.decoder_v(x)

        x_p = x_p.permute(1,0,2)
        x_v = x_v.permute(1,0,2)

        x_p = self.p_out(x_p)
        x_v = self.v_out(x_v)

        return x_p, x_v

In [5]:
'''
Single-Car Training
Normalization: /4800, /100
'''

import time
from tqdm import tqdm

batch_sz = 100
train_loader = DataLoader(train_dataset, batch_size=batch_sz, 
                          shuffle=False, collate_fn=my_collate_train, num_workers=2)

train_size = int(0.8 * len(train_loader.dataset))
val_size = len(train_loader.dataset) - train_size
train_data, val_data = torch.utils.data.random_split(train_loader.dataset, [train_size, val_size])

print("\nLENGTH OF TRAIN LOADER DATASET:", len(train_loader.dataset))
print("LENGTH OF TRAIN DATA:", len(train_data), "\nLENGTH OF VAL DATA:", len(val_data))

train_data = DataLoader(train_data, batch_size=batch_sz, 
                        shuffle=False, collate_fn=my_collate_train, num_workers=2)

val_data = DataLoader(val_data, batch_size=batch_sz, 
                      shuffle=False, collate_fn=my_collate_train, num_workers=2)

model = Trajectory().to(device)

my_optim = torch.optim.Adam(model.parameters())

epoch = 20 # takes around 20 epochs to converge
number = 1 # number of cars in each 

for i in range(epoch):
    model.train()
    epoch_loss = 0
    
    train_loop = tqdm(enumerate(train_data), total=len(train_data))
    
    for i_batch, sample_batch in train_loop:
        inp, out = sample_batch
        batch = inp.shape[0]

        p_in = (inp[:,0].reshape(batch*number,19,2).to(device))/4800. # -2400.)/2400.
        v_in = inp[:,1].reshape(batch*number,19,2).to(device)/100.
        p_out = (out[:,0].reshape(batch*number,30,2).to(device))/4800. # -2400.)/2400.
        v_out = out[:,1].reshape(batch*number,30,2).to(device)/100.
        pred = model(p_in, v_in)

        loss = 0
        p_criteria = nn.MSELoss()
        p_loss = torch.sqrt(p_criteria(pred[0], p_out))

        v_criteria = nn.MSELoss()
        v_loss = torch.sqrt(v_criteria(pred[1], v_out))

        loss = p_loss + v_loss
        epoch_loss += p_loss

        my_optim.zero_grad()
        loss.backward()
        my_optim.step()
    
        # update progress bar
        train_loop.set_description(f"Train Epoch [{i + 1}/{epoch}]")
        train_loop.set_postfix(loss = epoch_loss.item())

    model.eval()
    epoch_loss = 0
    
    val_loop = tqdm(enumerate(val_data), total=len(val_data))
    
    for i_batch, sample_batch in val_loop:
        inp, out = sample_batch
        batch = inp.shape[0]

        p_in = (inp[:,0].reshape(batch*number,19,2).to(device))/4800.
        v_in = inp[:,1].reshape(batch*number,19,2).to(device)/100.
        p_out = (out[:,0].reshape(batch*number,30,2).to(device))/4800. 
        v_out = out[:,1].reshape(batch*number,30,2).to(device)/100.

        pred = model(p_in, v_in)

        loss = 0
        p_criteria = nn.MSELoss()
        p_loss = torch.sqrt(p_criteria(pred[0], p_out))

        epoch_loss += p_loss
        
        val_loop.set_description(f"Val.  Epoch [{i + 1}/{epoch}]")
        val_loop.set_postfix(loss = epoch_loss.item())


LENGTH OF TRAIN LOADER DATASET: 205942
LENGTH OF TRAIN DATA: 164753 
LENGTH OF VAL DATA: 41189


Val.  Epoch [20/20]: 100%|██████████| 412/412 [00:14<00:00, 28.64it/s, loss=0.765]


In [6]:
'''
Single-Car Training
Normalization: /2400, /85
'''

import time
from tqdm import tqdm

batch_sz = 100
train_loader = DataLoader(train_dataset, batch_size=batch_sz, 
                          shuffle=False, collate_fn=my_collate_train, num_workers=2)

train_size = int(0.8 * len(train_loader.dataset))
val_size = len(train_loader.dataset) - train_size
train_data, val_data = torch.utils.data.random_split(train_loader.dataset, [train_size, val_size])

print("\nLENGTH OF TRAIN LOADER DATASET:", len(train_loader.dataset))
print("LENGTH OF TRAIN DATA:", len(train_data), "\nLENGTH OF VAL DATA:", len(val_data))

train_data = DataLoader(train_data, batch_size=batch_sz, 
                        shuffle=False, collate_fn=my_collate_train, num_workers=2)

val_data = DataLoader(val_data, batch_size=batch_sz, 
                      shuffle=False, collate_fn=my_collate_train, num_workers=2)

model = Trajectory().to(device)

my_optim = torch.optim.Adam(model.parameters())

epoch = 20 # takes around 20 epochs to converge
number = 1 # number of cars in each 

for i in range(epoch):
    model.train()
    epoch_loss = 0
    
    train_loop = tqdm(enumerate(train_data), total=len(train_data))
    
    for i_batch, sample_batch in train_loop:
        inp, out = sample_batch
        batch = inp.shape[0]

        p_in = ((inp[:,0].reshape(batch*number,19,2).to(device)) - 2400.) / 2400.
        v_in = inp[:,1].reshape(batch*number,19,2).to(device)/85.
        p_out = ((out[:,0].reshape(batch*number,30,2).to(device)) - 2400.) / 2400.
        v_out = out[:,1].reshape(batch*number,30,2).to(device)/85.
        pred = model(p_in, v_in)

        loss = 0
        p_criteria = nn.MSELoss()
        p_loss = torch.sqrt(p_criteria(pred[0], p_out))

        v_criteria = nn.MSELoss()
        v_loss = torch.sqrt(v_criteria(pred[1], v_out))

        loss = p_loss + v_loss
        epoch_loss += p_loss

        my_optim.zero_grad()
        loss.backward()
        my_optim.step()
    
        # update progress bar
        train_loop.set_description(f"Train Epoch [{i + 1}/{epoch}]")
        train_loop.set_postfix(loss = epoch_loss.item())

    model.eval()
    epoch_loss = 0
    
    val_loop = tqdm(enumerate(val_data), total=len(val_data))
    
    for i_batch, sample_batch in val_loop:
        inp, out = sample_batch
        batch = inp.shape[0]

        p_in = ((inp[:,0].reshape(batch*number,19,2).to(device)) - 2400.)/2400.
        v_in = inp[:,1].reshape(batch*number,19,2).to(device)/85.
        p_out = ((out[:,0].reshape(batch*number,30,2).to(device)) - 2400.)/2400. 
        v_out = out[:,1].reshape(batch*number,30,2).to(device)/85.

        pred = model(p_in, v_in)

        loss = 0
        p_criteria = nn.MSELoss()
        p_loss = torch.sqrt(p_criteria(pred[0], p_out))

        epoch_loss += p_loss
        
        val_loop.set_description(f"Val.  Epoch [{i + 1}/{epoch}]")
        val_loop.set_postfix(loss = epoch_loss.item())


LENGTH OF TRAIN LOADER DATASET: 205942
LENGTH OF TRAIN DATA: 164753 
LENGTH OF VAL DATA: 41189


Val.  Epoch [20/20]: 100%|██████████| 412/412 [00:15<00:00, 26.11it/s, loss=0.994]


- Training Loss = 2.52
- Validation Loss = 0.762

In [ ]:
state = {
    'epoch': epoch,
    'state_dict': model.state_dict(),
    'optimizer': my_optim.state_dict(),
}
torch.save(state, "saved_model/PogNet1.pt")

In [5]:
import time
from tqdm import tqdm

batch_sz = 100
train_loader = DataLoader(train_dataset, batch_size=batch_sz, 
                          shuffle=False, collate_fn=my_collate_train, num_workers=2)


print("\nLENGTH OF TRAIN LOADER DATASET:", len(train_loader.dataset))

model = Trajectory().to(device)

my_optim = torch.optim.Adam(model.parameters())

epoch = 20 # takes around 20 epochs to converge
number = 1 # number of cars in each 

for i in range(epoch):
    model.train()
    epoch_loss = 0
    
    train_loop = tqdm(enumerate(train_loader), total=len(train_loader))
    
    for i_batch, sample_batch in train_loop:
        inp, out = sample_batch
        batch = inp.shape[0]

        p_in = (inp[:,0].reshape(batch*number,19,2).to(device))/4800. # -2400.)/2400.
        v_in = inp[:,1].reshape(batch*number,19,2).to(device)/100.
        p_out = (out[:,0].reshape(batch*number,30,2).to(device))/4800. # -2400.)/2400.
        v_out = out[:,1].reshape(batch*number,30,2).to(device)/100.
        pred = model(p_in, v_in)

        loss = 0
        p_criteria = nn.MSELoss()
        p_loss = torch.sqrt(p_criteria(pred[0], p_out))

        v_criteria = nn.MSELoss()
        v_loss = torch.sqrt(v_criteria(pred[1], v_out))

        loss = p_loss + v_loss
        epoch_loss += p_loss

        my_optim.zero_grad()
        loss.backward()
        my_optim.step()
    
        # update progress bar
        train_loop.set_description(f"Train Epoch [{i + 1}/{epoch}]")
        train_loop.set_postfix(loss = epoch_loss.item())

    model.eval()
    epoch_loss = 0


LENGTH OF TRAIN LOADER DATASET: 205942


Train Epoch [20/20]: 100%|██████████| 2060/2060 [00:45<00:00, 44.81it/s, loss=2.89]


In [ ]:
import matplotlib.pyplot as plt

model.eval()

fig, (plt1, plt2, plt3) = plt.subplots(3)

for i_batch, sample_batch in tqdm(enumerate(train_loader), total=len(train_loader)):
    inp, out = sample_batch
    batch = inp.shape[0]

    p_in = (inp[:,0].reshape(batch*number,19,2).to(device))/4800. # -2400.)/2400.
    v_in = inp[:,1].reshape(batch*number,19,2).to(device)/100.
    p_out = (out[:,0].reshape(batch*number,30,2).to(device))/4800. # -2400.)/2400.
    v_out = out[:,1].reshape(batch*number,30,2).to(device)/100.

    pred = model(p_in, v_in)
    
    x = pred[0][10]*4800
    y = p_out[10]*4800
    z = torch.subtract(x, y)
    print(x)
    print(y)
    print(torch.sum(z))
    
    plt1.plot(x.cpu().detach().numpy(), y.cpu().numpy())
    plt2.plot(y.cpu().numpy())
    plt3.plot(z.cpu().detach().numpy())
    break


In [ ]:
'''
Visualize Trajectories
'''

import matplotlib.pyplot as plt
import random

def show_sample_batch(sample_batch, agent_id):
    """visualize the trajectory for a batch of samples with a random agent"""
    inp, out = sample_batch
    batch_sz = inp.size(0)
    
    p_in = (inp[:,0].reshape(batch_sz * number, 19, 2).to(device))/4800. # -2400.)/2400.
    v_in = inp[:,1].reshape(batch_sz * number, 19, 2).to(device)/100.
    p_out = (out[:,0].reshape(batch_sz * number, 30, 2).to(device))/4800. # -2400.)/2400.
    v_out = out[:,1].reshape(batch_sz * number, 30, 2).to(device)/100.

    pred = model(p_in, v_in)
    print(pred[0].shape)
    x = (pred[0][10]*4800).cpu().detach().numpy()
    y = (p_out[10]*4800).cpu().numpy()
    exit()    
    fig, axs = plt.subplots(10, 10, figsize=(50, 50), facecolor='w', edgecolor='k')
    fig.subplots_adjust(hspace = .5, wspace=.001)
    axs = axs.ravel()   
    for i in range(batch_sz):
        axs[i].xaxis.set_ticks([])
        axs[i].yaxis.set_ticks([])
        
        # first two feature dimensions are (x,y) positions
        axs[i].scatter(inp[i, agent_id,:,0], inp[i, agent_id,:,1], c="blue")
        axs[i].scatter(out[i, agent_id,:,0], out[i, agent_id,:,1], c="red")
        axs[i].scatter(x, y, c="black")

model.eval()
agent_id = 0
     
for i_batch, sample_batch in enumerate(train_loader):
    inp, out = sample_batch
    show_sample_batch(sample_batch, agent_id)
    break

In [6]:
import csv 
model.eval()
temp = []

new_path = "data/new_val_in/"
val_dataset  = ArgoverseDataset(data_path=new_path)

top = []
top.append("ID")
for i in range(60):
    top.append("v"+str(i+1))
temp.append(top)

with torch.no_grad():
    for i in val_dataset:
        row = []
        scene = i['scene_idx']
        agent = i['agent_id']
        target =0
        for x in range(len(i['track_id'])):
            if i['track_id'][x][0] == agent:
                target = x

        p_in = torch.LongTensor(i['p_in'])
        v_in = torch.LongTensor(i['v_in'])



        p_in = (p_in.reshape(60,19,2).to(device))/4800.
        v_in = v_in.reshape(60,19,2).to(device)/100.

        pred = model(p_in, v_in)

        pred_out = pred[0]*4800.
      
        output = pred_out[target]

        row.append(scene)
        row = row + torch.flatten(output).cpu().detach().numpy().tolist()
        temp.append(row)

with open('submission9.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(temp)